In [1]:
import pandas as pd
import re

# Data Cleaning Steps

1. Create Unique ID for reviews/news/posts
2. Remove special characters
3. Lemmatization
4. Remove stopwords
5. TFIDF Vectorization

## 1. Cleaning trustpilot data

### Create Unique ID

In [2]:
df= pd.read_csv(r'Data\Uber_Trustpilot_Review.csv')
df.sample(5)

,review_page_number,reviewer_name,reviewer_review_count,review_location,star_rating,review_title,review,review_month,review_dat,review_year,review_source,company_name,company_website,company_website_html,total_reviews,avg_star_rating,total_review_pages
5054,773,Ghassan Hamad,1,US,1,stay a way from Uber,"Uber company they don't respect the drivers , ...",July,14,2018,Trustpilot,Uber,www.uber.com,http://www.uber.com?utm_medium=company_profile...,23919,3.0,811
6236,69,Armin Hakimi,7,GB,1,Absolutely untrustworthy company,Absolutely untrustworthy company . I left an e...,July,22,2023,Trustpilot,Uber,www.uber.com,http://www.uber.com?utm_medium=company_profile...,23919,3.0,811
214,15,diane vaughan,3,US,1,I just tried to get a price and I can't…,I just tried to get a price and I can't even g...,December,26,2023,Trustpilot,Uber,www.uber.com,http://www.uber.com?utm_medium=company_profile...,23919,3.0,811
13244,452,Rebecca Zubia,1,US,5,Diane was a wonderful person and a very…,Diane was a wonderful person and a very safe a...,July,27,2022,Trustpilot,Uber,www.uber.com,http://www.uber.com?utm_medium=company_profile...,23919,3.0,811
3026,604,Carmela DeNicola,2,US,1,Scheduled a ride from my home to the…,Scheduled a ride from my home to the Philly ai...,May,26,2021,Trustpilot,Uber,www.uber.com,http://www.uber.com?utm_medium=company_profile...,23919,3.0,811


In [3]:
df.shape

(16207, 17)

In [4]:
df= df.reset_index()
df.rename(columns={'index':'UID'}, inplace= True)
df['UID']= ["TP_"+str(i) for i in df['UID']]
df.sample(3)

,UID,review_page_number,reviewer_name,reviewer_review_count,review_location,star_rating,review_title,review,review_month,review_dat,review_year,review_source,company_name,company_website,company_website_html,total_reviews,avg_star_rating,total_review_pages
14970,TP_14970,643,Karim I.,3,CA,1,Worst tech support in the industry,Their tech support is the worst in the industr...,December,21,2020,Trustpilot,Uber,www.uber.com,http://www.uber.com?utm_medium=company_profile...,23919,3.0,811
7172,TP_7172,118,Bek Ara,1,US,5,The best app in the world,NaN,May,22,2023,Trustpilot,Uber,www.uber.com,http://www.uber.com?utm_medium=company_profile...,23919,3.0,811
5307,TP_5307,790,Mr Lee Campbell,4,GB,1,Too expensive,Uber was daylight robbery as far as I am conce...,October,1,2017,Trustpilot,Uber,www.uber.com,http://www.uber.com?utm_medium=company_profile...,23919,3.0,811


### Filter out reviews since 2021 in country United States

In [5]:
df= df[(df["review_year"]>= 2021) & (df["review_location"]== "US")]
df.shape

(6015, 18)

### Sepearate ID, reviews and sentiment

In [8]:
UID= df['UID'].tolist()
reviews= df['review'].tolist()
sentiment= df['star_rating'].apply(lambda x: "Negative" if x<= 3 else "Postitive").tolist()

print(len(UID))
print(len(reviews))
print(len(sentiment))

6015
6015
6015


In [9]:
df[df['UID']=="NaN"]

,UID,review_page_number,reviewer_name,reviewer_review_count,review_location,star_rating,review_title,review,review_month,review_dat,review_year,review_source,company_name,company_website,company_website_html,total_reviews,avg_star_rating,total_review_pages


### Remove Special Characters (punctuations and emoji) and numbers

In [253]:
'''
import emoji
def remove_emoji (text):
    emoji_check= emoji.emoji_list(text) #("adsas 🤔 🤔")
    if (emoji_check==[]):
        return text 
    else:
        result=""
        for char in text:
            if(emoji.is_emoji(char)):
                continue
            else:
                result+=char
        return result
'''


'\nimport emoji\ndef remove_emoji (text):\n    emoji_check= emoji.emoji_list(text) #("adsas 🤔 🤔")\n    if (emoji_check==[]):\n        return text \n    else:\n        result=""\n        for char in text:\n            if(emoji.is_emoji(char)):\n                continue\n            else:\n                result+=char\n        return result\n'

In [10]:
def remove_emoji_punctuations(text):
    text= str(text)
     # Pattern to only match letters, and whitespaces.
    pattern = re.compile(r'[\W\d\s]+')
    #punctuation_pattern = re.compile(r'[^\w\s\d]')
    # Remove punctuations if match found
    return pattern.sub(' ', text)

reviews_without_spl_char= [remove_emoji_punctuations(i) for i in reviews]
len(reviews_without_spl_char)

6015

In [11]:
# Verification 

test= pd.DataFrame({
    "Review_Original": reviews,
    "Review_Without_Spl_Char": reviews_without_spl_char
})

test[test['Review_Original']!=test['Review_Without_Spl_Char']].to_excel("After Removing Spl Char Trustpilot.xlsx", index=False)

### Lemmatization and lower text

In [12]:
len(UID)
len(reviews_without_spl_char)

6015

In [13]:
reviews_without_spl_char[2]

'I was shown a price if and charged twice at first for this trip Which created a terrible morning experience for my grandmother seeing how this was a trip to get me something to smoke and grab her breakfast Uber was not as helpful or convenient as I usually find the service And I used to drive for the company In the end I was charged I just feel lied to is all '

In [14]:
#! pip install nltk
#import nltk
#nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

In [15]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("rocks")

def lemmatize(text):
    result=""
    for word in text.split():
        result+=lemmatizer.lemmatize(word.lower())+" "
    return result.strip()

reviews_lemmatized= [lemmatize(i) for i in reviews_without_spl_char]
len(reviews_lemmatized)

6015

In [16]:
# Verification 

test= pd.DataFrame({
    "reviews_without_spl_char": reviews_without_spl_char,
    "reviews_lemmatized": reviews_lemmatized
})

test[test['reviews_without_spl_char']!=test['reviews_lemmatized']].to_excel("After Lemmatizing V2.xlsx", index=False)

### Remove Stopwords

In [17]:
from nltk.corpus import stopwords

In [18]:
# Define stop word removal function and call it for the lemmatized text

nltk_stopwords= stopwords.words('english')
self_stopwords= ['uber', 'wa','go','tell','didnt','one','driver']
total_stopwords = nltk_stopwords + self_stopwords
print("Total Number of stop words:", len(total_stopwords))

def remove_stop_words(text):
    result=""
    for word in text.split():
        if(word not in total_stopwords):
            result+=word+" "
    return result.strip()

reviews_without_stop_words= [remove_stop_words(i) for i in reviews_lemmatized]
len(reviews_without_stop_words)

Total Number of stop words: 186


6015

In [19]:
# Verification 

test= pd.DataFrame({
    "reviews_with_stop_words": reviews_lemmatized,
    "reviews_without_stop_words": reviews_without_stop_words
})

test[test['reviews_with_stop_words']!=test['reviews_without_stop_words']].to_excel("After Removing Stop Words.xlsx", index=False)

### Word Cloud Comparision

In [20]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Sample text data
text = "".join(str(i) for i in reviews)

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud Before Cleaning")
plt.show()


In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Sample text data
text = "".join(reviews_without_stop_words)

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud After Cleaning")
plt.show()


### TFIDF Vectorization

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
input= 'content'
max_features= 500
max_df= 0.7 # Ignores the words with frequnce more than this number- Usefull to remove content specific stop words.

vectorizer= TfidfVectorizer(input=input, max_df=max_df, max_features=max_features)
doc_term= vectorizer.fit_transform(reviews_without_stop_words)

In [24]:
doc_term.shape

(6015, 500)

In [25]:
vectorizer.get_feature_names_out()

array(['able', 'absolutely', 'accept', 'accepted', 'account', 'actually',
       'add', 'address', 'advance', 'ago', 'ahead', 'airport', 'allow',
       'almost', 'already', 'also', 'always', 'amazing', 'amount',
       'another', 'answer', 'anymore', 'anyone', 'anything', 'app',
       'appointment', 'appreciate', 'area', 'around', 'arrival', 'arrive',
       'arrived', 'ask', 'asked', 'asking', 'attempt', 'available',
       'away', 'awesome', 'back', 'background', 'bad', 'bag', 'bank',
       'best', 'better', 'big', 'bit', 'black', 'block', 'book', 'booked',
       'booking', 'business', 'busy', 'cab', 'call', 'called', 'calling',
       'came', 'cancel', 'canceled', 'cancellation', 'cancelled',
       'cannot', 'car', 'card', 'care', 'case', 'cash', 'cause', 'change',
       'changed', 'charge', 'charged', 'charging', 'chat', 'cheaper',
       'check', 'city', 'clean', 'close', 'cold', 'come', 'comfortable',
       'coming', 'communication', 'company', 'complaint', 'complete',
   

In [26]:
doc_term_df= pd.DataFrame(doc_term.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_df['UID']= UID
# Make UID as the first column
doc_term_df= doc_term_df[['UID']+list(doc_term_df.columns[:-1])]
doc_term_df.sample(4)

,UID,able,absolutely,accept,accepted,account,actually,add,address,advance,...,working,worse,worst,worth,would,wrong,year,yesterday,yet,zero
2017,TP_7354,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.169073,0.0,0.0
643,TP_2182,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1486,TP_6465,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5884,TP_14717,0.0,0.0,0.0,0.0,0.0,0.159306,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [27]:
# Saving the vectorizer and doc_term matrix for later use

from scipy import sparse
sparse.save_npz('doc_term_sparse.npz', doc_term)
#loaded_doc_term = sparse.load_npz('doc_term_sparse.npz')

import pickle
# Save the vectorizer to a file
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
"""
# Load the vectorizer from the file
with open('vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)
"""

"\n# Load the vectorizer from the file\nwith open('vectorizer.pkl', 'rb') as f:\n    loaded_vectorizer = pickle.load(f)\n"

### Combining the doc term with sentiment

In [29]:
doc_term_df['Sentiment']= sentiment
doc_term_df.head()

,UID,able,absolutely,accept,accepted,account,actually,add,address,advance,...,worse,worst,worth,would,wrong,year,yesterday,yet,zero,Sentiment
0,TP_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.263133,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Negative
1,TP_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Postitive
2,TP_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Negative
3,TP_13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Negative
4,TP_14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.108582,0.0,0.0,0.0,0.0,0.0,Negative


### Save the dataframe

In [31]:
doc_term_df.to_csv('Data\Cleaned_Data\TrustPilot_Cleaned_Doc_Term_With_Sentiment.csv', index=False)